<a href="https://colab.research.google.com/github/PHATTARAWAT2312/ConcealedGun/blob/main/3_Tracking_Model_Predict_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tracking Model_Predict-AllVidio

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#root_path = 'gdrive/My Drive/ConcealedGunPROJECT2020/'  #change dir to your project folder

Mounted at /content/gdrive


In [2]:
import pandas as pd
# import all the tools we need
import urllib
import pandas as pd
import numpy as np
import cv2
import itertools
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import os 
from PIL import Image
import random
import xml.etree.ElementTree as ET
import time
import requests
from pandas import DataFrame
import glob
import json
import subprocess
import pickle
from sklearn.preprocessing import StandardScaler
import shutil
import pathlib
from statistics import mean 
from matplotlib import pyplot as plt

### รวม csv ทุกอันที่ได้จาก Result_Test_Tracking

In [5]:
data_videoname1 = pd.read_csv("/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/Name_Pass-Result_Test_Tracking.csv")
data_videoname2 = []
for i in range(data_videoname1.shape[0]):
    x = data_videoname1.iloc[i]['Clip_Name'].split(' ')
    x = x[0].replace('[]','')
    data_videoname2.append(x)
data_all = []
location = "/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/Result_Test_Tracking/"
data_all = [pd.read_csv (f'{location}{s}.csv',parse_dates=True) for s in data_videoname2]
data_all

[    idx_frame  idx_per  idx_per_next  ... PersonNo ClassMode  ProbMean
 0           0        0             0  ...        0         1  0.948527
 1           1        0             0  ...        0         1  0.948527
 2           2        0             0  ...        0         1  0.948527
 3           3        0             0  ...        0         1  0.948527
 4           4        0             0  ...        0         1  0.948527
 5           5        0             0  ...        0         1  0.948527
 6           6        0             0  ...        0         1  0.948527
 7           7        0             0  ...        0         1  0.948527
 8           8        0             0  ...        0         1  0.948527
 9           9        0             0  ...        0         1  0.948527
 10          0        0             0  ...        0         1  0.948527
 11          1        0             0  ...        0         1  0.948527
 12          2        0             0  ...        0         1  0

In [6]:
df_all = pd.concat(data_all)
df_all

,idx_frame,idx_per,idx_per_next,test_img,test_box,prob,pred,VideoName,PersonNo,ClassMode,ProbMean
0,0,0,0,[[[0.92941177 0.9529412 0.94509804]\n [0.960...,[ 0. 31.047058 176.6407 456.6067 ],0.909810,1,zg9ct7cWUWw_dup1,0,1,0.948527
1,1,0,0,[[[0.92941177 0.9529412 0.9372549 ]\n [0.960...,[ 37.895367 16.318016 198.87242 458.80463 ],0.940064,2,zg9ct7cWUWw_dup1,0,1,0.948527
2,2,0,0,[[[0.9254902 0.94509804 0.92941177]\n [0.960...,[ 97.57488 17.807297 281.89075 452.60516 ],0.949703,1,zg9ct7cWUWw_dup1,0,1,0.948527
3,3,0,0,[[[0.9254902 0.94509804 0.92941177]\n [0.960...,[115.983215 14.840234 317.3526 446.31467 ],0.956157,1,zg9ct7cWUWw_dup1,0,1,0.948527
4,4,0,0,[[[0.9254902 0.94509804 0.92941177]\n [0.960...,[ 97.996704 17.092533 282.2711 452.30014 ],0.949024,1,zg9ct7cWUWw_dup1,0,1,0.948527
...,...,...,...,...,...,...,...,...,...,...,...
65,25,1,1,[[[0.91764706 0.9019608 0.96862745]\n [0.913...,[230.34503 5.0445237 342.90387 170.15904 ],0.927916,2,_0fBCjD6Dhc,1,2,0.902259
66,26,1,1,[[[0.78431374 0.7764706 0.81960785]\n [0.788...,[233.97784 1.4710395 338.63748 168.2514 ],0.917611,2,_0fBCjD6Dhc,1,2,0.902259
67,27,1,1,[[[0.78431374 0.7764706 0.81960785]\n [0.788...,[234.0446 1.5486574 338.6224 167.92064 ],0.909092,2,_0fBCjD6Dhc,1,2,0.902259
68,28,1,1,[[[0.91764706 0.9019608 0.96862745]\n [0.913...,[231.34946 4.9963512 342.76447 169.94273 ],0.924369,2,_0fBCjD6Dhc,1,2,0.902259


### จำนวนคนสำหรับ Test set

In [7]:
group_per_name = df_all.groupby(['VideoName','PersonNo']).size().reset_index(name='counts_frame')
group_per_name

,VideoName,PersonNo,counts_frame
0,-8zyEwAa50Q_dup3,0,80
1,-8zyEwAa50Q_dup3,1,70
2,-8zyEwAa50Q_dup3,2,70
3,-8zyEwAa50Q_dup4,0,60
4,-8zyEwAa50Q_dup4,1,50
...,...,...,...
153,waGa7d7NEtQ_dup2,0,80
154,waGa7d7NEtQ_dup2,1,70
155,xieJf8Wa1kc,0,70
156,xieJf8Wa1kc,1,60


### ทำนายผลทั้งคลิปวิดีโอ โดยดูจากค่า Mode 
> เช่น วิดีโอ -8zyEwAa50Q_dup3 คนที่ 0 มี 80 เฟรม
แสดงว่ามี 8 คำตอบ 


> ดังนั้นโปรแกรมจะดูว่า ใน 8 คำตอบมี Class ไหนมากกว่านั้นก็จะให้แสดงเป็น Class นั้น

In [99]:
fs_all = []

In [110]:
idclip = str(input("ID Clip : "))
# Ex. QP57keNgs4o_dup1
idclip_csv = idclip+'.csv'
df = pd.read_csv('/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/Result_Test_Tracking/'+idclip_csv) #**************** 1 ****************
group_per_df0 = df0.groupby(['PersonNo']).size().reset_index(name='counts_frame')
# print(group_per_df0)
test = pd.read_csv ('/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun/DATASET.csv')
group_per = test.groupby(['Clip_Name','person_id','people','class_id']).size().reset_index(name='counts_frame')
asd = group_per[group_per['Clip_Name'] ==idclip]
num_per = int(len(asd))
num_per
for i in range(num_per):
  df0 = df[df['PersonNo'] ==i]        
  newdf = df0['ProbMean'].agg(['mean']) # ค่า mean ของ Prob 
  scor = newdf*100
  scor = float(scor) 
  newdf0 = df0['ClassMode'].agg(['mode']) # ค่า Mode ของ class
  newdf0['VideoName'] = idclip
  newdf0['Scor'] = scor
  asd0= asd['person_id'][0+i:1+i] # คนที่ 0
  asd0 = int(asd0) 
  newdf0['person_id'] = asd0
  f0 = newdf0
  fs_all.append(f0)
fs_all

ID Clip : waGa7d7NEtQ_dup2


[   mode         VideoName       Scor  person_id
 0     2  QP57keNgs4o_dup1  99.222685     106687,
    mode         VideoName       Scor  person_id
 0     2  QP57keNgs4o_dup1  89.360479     106688,
    mode         VideoName       Scor  person_id
 0     1  -8zyEwAa50Q_dup3  99.052549     119847,
    mode         VideoName       Scor  person_id
 0     1  -8zyEwAa50Q_dup3  99.017789     119848,
    mode         VideoName       Scor  person_id
 0     2  waGa7d7NEtQ_dup2  99.693552     111475,
    mode         VideoName       Scor  person_id
 0     1  waGa7d7NEtQ_dup2  98.793303     111477]

In [113]:
result_end = pd.concat(fs_all)

In [114]:
# os.chdir('/content/gdrive/MyDrive/CONEX/CONEX_ConcealedGun')
# result_end_s.to_csv('End_Result_Test_Tracking.csv', index = False) 

# Completed ผลลัพธ์ที่ได้ คือ
1. คำตอบสำหรับ 1 คน (Class(mode), VideoName, Scor, person_id )
2. นำ End_Result_Test_Tracking.csv ไปหา Confision Matrix